In [1]:
%load_ext autoreload
%autoreload 


In [3]:
import sys
import subprocess

sys.path.append("../qiskit-qec/src/")  # adjust the path as needed

from qiskit_qec.circuits import ArcCircuit


In [2]:
from qiskit_qec.circuits import ArcCircuit
from qiskit_qec.decoders import UnionFindDecoder
from qiskit_aer import AerSimulator

from qiskit.providers.fake_provider import FakeSherbrooke

We'll consider a repetition code on the following line of qubits (ancillas in brackets). In this example, we'll consider an encoded logical `1`.

```
    0--(1)--2--(3)--4--(5)--6--(7)--8
```

In [13]:
links = [[0, 1, 2], [2, 3, 4], [4, 5, 6], [6, 7, 8]] # q1, a, q2 
T = 5 # number of syndrome measurement rounds
code = ArcCircuit(links, T, basis='zz',logical='1') 

Let's look at the circuit.

In [14]:
qc = code.circuit['zz']
qc.draw(fold=-1)

░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                        
      link_qubit_0: ──────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├────────────────────────
                          ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘┌─┐                     
      link_qubit_1: ──────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫─┤M├─────────────────────
                          ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║ └╥┘┌─┐                  
      link_qubit_2: ──────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫──╫─┤M├──────────────────
                          ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║  ║ └╥┘┌─┐               
      link_qubit_3: ──────░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫──╫──╫─┤M├───────────────
                    ┌───┐ ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║  ║  ║ └╥┘┌─┐            
      code_qubit_0: ┤ X ├─░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫──╫──╫──╫─┤M├────────────
                    ├───┤ ░   │         │    │    │    │    │    │   ░  ║   ║    ║    ║        ░   │         │    │    │    │    │    │   ░  ║   ║    ║    ║        ░   │         │    │    │    │    │    │   ░  ║   ║    ║    ║        ░   │         │    │    │    │    │    │   

By running a noiseless simulation, we can see what a noiseless result looks like.

In [15]:
AerSimulator().run(qc).result().get_counts()

{'11111 0000 0000 0000 0000 0000': 1024}

We'll set up a Union Find decoder to use with this code.

In [16]:
decoder = UnionFindDecoder(code)

Let's first try it on the noiseless outcome.

In [17]:
decoder.process('11111 0000 0000 0000 0000 0000')


ValueError: {'is_boundary': True, 'time': None, 'qubits': [0], 'index': 0, 'properties': {'conjugate': False, 'link qubit': None}} is not in list

The result here tells use that the result of the logical measurement is `1`, as required.

Now let's get noisy results. For example: from a fake backend.

In [26]:
counts = FakeSherbrooke().run(qc, shots=10000).result().get_counts()

We can run this through the decoder to see how many times we get logical 0 and logical 1.

In [27]:
logical_counts = {0:0, 1:0}
for string, shots in counts.items():
    logical_counts[decoder.process(string)[0]] += shots
logical_counts

ValueError: {'is_boundary': True, 'time': None, 'qubits': [0], 'index': 0, 'properties': {'conjugate': False, 'link qubit': None}} is not in list

The numbers will change with shot noise, but the correct outcome (logical 1) certainly seems to come out on top!